In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [ ]:
data = pd.read_csv("/content/LS_2.0.csv")
data.head()

,STATE,CONSTITUENCY,NAME,WINNER,PARTY,SYMBOL,GENDER,CRIMINAL\nCASES,AGE,CATEGORY,EDUCATION,ASSETS,LIABILITIES,GENERAL\nVOTES,POSTAL\nVOTES,TOTAL\nVOTES,OVER TOTAL ELECTORS \nIN CONSTITUENCY,OVER TOTAL VOTES POLLED \nIN CONSTITUENCY,TOTAL ELECTORS
0,Telangana,ADILABAD,SOYAM BAPU RAO,1,BJP,Lotus,MALE,52,52.0,ST,12th Pass,"Rs 30,99,414\n ~ 30 Lacs+","Rs 2,31,450\n ~ 2 Lacs+",376892,482,377374,25.330684,35.468248,1489790
1,Telangana,ADILABAD,Godam Nagesh,0,TRS,Car,MALE,0,54.0,ST,Post Graduate,"Rs 1,84,77,888\n ~ 1 Crore+","Rs 8,47,000\n ~ 8 Lacs+",318665,149,318814,21.399929,29.964370,1489790
2,Telangana,ADILABAD,RATHOD RAMESH,0,INC,Hand,MALE,3,52.0,ST,12th Pass,"Rs 3,64,91,000\n ~ 3 Crore+","Rs 1,53,00,000\n ~ 1 Crore+",314057,181,314238,21.092771,29.534285,1489790
3,Telangana,ADILABAD,NOTA,0,NOTA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13030,6,13036,0.875023,1.225214,1489790
4,Uttar Pradesh,AGRA,Satyapal Singh Baghel,1,BJP,Lotus,MALE,5,58.0,SC,Doctorate,"Rs 7,42,74,036\n ~ 7 Crore+","Rs 86,06,522\n ~ 86 Lacs+",644459,2416,646875,33.383823,56.464615,1937690


In [ ]:
def correlation(data, threshold):
  col_corr = set()
  corr_matrix = data.corr()
  for i in range(len(corr_matrix.columns)):
    for j in range(i):
      if abs(corr_matrix.iloc[i,j]>threshold):
        col_name = corr_matrix.columns[i]
        col_corr.add(col_name)
  return col_corr

corr_features = correlation(data,0.80)
len(set(corr_features))
corr_features 

{'OVER TOTAL ELECTORS \nIN CONSTITUENCY',
 'OVER TOTAL VOTES POLLED \nIN CONSTITUENCY',
 'TOTAL\nVOTES'}

In [ ]:
data.isnull().sum().sum()

1960

In [ ]:
data.isnull().sum()

STATE                                          0
CONSTITUENCY                                   0
NAME                                           0
WINNER                                         0
PARTY                                          0
SYMBOL                                       245
GENDER                                       245
CRIMINAL\nCASES                              245
AGE                                          245
CATEGORY                                     245
EDUCATION                                    245
ASSETS                                       245
LIABILITIES                                  245
GENERAL\nVOTES                                 0
POSTAL\nVOTES                                  0
TOTAL\nVOTES                                   0
OVER TOTAL ELECTORS \nIN CONSTITUENCY          0
OVER TOTAL VOTES POLLED \nIN CONSTITUENCY      0
TOTAL ELECTORS                                 0
dtype: int64

In [ ]:
data.columns

Index(['STATE', 'CONSTITUENCY', 'NAME', 'WINNER', 'PARTY', 'SYMBOL', 'GENDER',
       'CRIMINAL\nCASES', 'AGE', 'CATEGORY', 'EDUCATION', 'ASSETS',
       'LIABILITIES', 'GENERAL\nVOTES', 'POSTAL\nVOTES', 'TOTAL\nVOTES',
       'OVER TOTAL ELECTORS \nIN CONSTITUENCY',
       'OVER TOTAL VOTES POLLED \nIN CONSTITUENCY', 'TOTAL ELECTORS'],
      dtype='object')

In [ ]:
data.shape

(2263, 19)

In [ ]:
data.drop(['GENERAL\nVOTES','POSTAL\nVOTES','TOTAL\nVOTES','OVER TOTAL ELECTORS \nIN CONSTITUENCY','OVER TOTAL VOTES POLLED \nIN CONSTITUENCY'],axis=1,inplace=True)

In [ ]:
data.shape

(2263, 14)

In [ ]:
data.drop(['SYMBOL','NAME'],axis=1,inplace=True)

In [ ]:
data.shape

(2263, 12)

In [ ]:
data.isnull().sum()

STATE                0
CONSTITUENCY         0
WINNER               0
PARTY                0
GENDER             245
CRIMINAL\nCASES    245
AGE                245
CATEGORY           245
EDUCATION          245
ASSETS             245
LIABILITIES        245
TOTAL ELECTORS       0
dtype: int64

In [ ]:
data.drop(data[data['PARTY']=='NOTA'].index,axis=0,inplace=True)

In [ ]:
data.shape

(2018, 12)

In [ ]:
data.isnull().sum().sum()

0

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2018 entries, 0 to 2261
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   STATE           2018 non-null   object 
 1   CONSTITUENCY    2018 non-null   object 
 2   WINNER          2018 non-null   int64  
 3   PARTY           2018 non-null   object 
 4   GENDER          2018 non-null   object 
 5   CRIMINAL
CASES  2018 non-null   object 
 6   AGE             2018 non-null   float64
 7   CATEGORY        2018 non-null   object 
 8   EDUCATION       2018 non-null   object 
 9   ASSETS          2018 non-null   object 
 10  LIABILITIES     2018 non-null   object 
 11  TOTAL ELECTORS  2018 non-null   int64  
dtypes: float64(1), int64(2), object(9)
memory usage: 205.0+ KB


In [ ]:
data.head()

,STATE,CONSTITUENCY,WINNER,PARTY,GENDER,CRIMINAL\nCASES,AGE,CATEGORY,EDUCATION,ASSETS,LIABILITIES,TOTAL ELECTORS
0,Telangana,ADILABAD,1,BJP,MALE,52,52.0,ST,12th Pass,"Rs 30,99,414\n ~ 30 Lacs+","Rs 2,31,450\n ~ 2 Lacs+",1489790
1,Telangana,ADILABAD,0,TRS,MALE,0,54.0,ST,Post Graduate,"Rs 1,84,77,888\n ~ 1 Crore+","Rs 8,47,000\n ~ 8 Lacs+",1489790
2,Telangana,ADILABAD,0,INC,MALE,3,52.0,ST,12th Pass,"Rs 3,64,91,000\n ~ 3 Crore+","Rs 1,53,00,000\n ~ 1 Crore+",1489790
4,Uttar Pradesh,AGRA,1,BJP,MALE,5,58.0,SC,Doctorate,"Rs 7,42,74,036\n ~ 7 Crore+","Rs 86,06,522\n ~ 86 Lacs+",1937690
5,Uttar Pradesh,AGRA,0,BSP,MALE,0,47.0,SC,Post Graduate,"Rs 13,37,84,385\n ~ 13 Crore+","Rs 2,22,51,891\n ~ 2 Crore+",1937690


In [ ]:
def converter (x):
  try:

    x=x.split('\n')[0]
    x=x.split()[1]
    x=x.replace(",","")
    x=float(x)
  except:
   x=0
  return x   

In [ ]:
data['ASSETS']= data['ASSETS'].apply(lambda x : converter(x))

In [ ]:
data['LIABILITIES']= data['LIABILITIES'].apply(lambda x : converter(x))

In [ ]:
data['ASSETS']

0       3.099414e+06
1       1.847789e+07
2       3.649100e+07
4       7.427404e+07
5       1.337844e+08
            ...     
2257    1.168500e+06
2258    4.890000e+06
2259    1.287852e+09
2260    9.036630e+08
2261    5.857733e+07
Name: ASSETS, Length: 2018, dtype: float64

In [ ]:
data.head()

,STATE,CONSTITUENCY,WINNER,PARTY,GENDER,CRIMINAL\nCASES,AGE,CATEGORY,EDUCATION,ASSETS,LIABILITIES,TOTAL ELECTORS
0,Telangana,ADILABAD,1,BJP,MALE,52,52.0,ST,12th Pass,3099414.0,231450.0,1489790
1,Telangana,ADILABAD,0,TRS,MALE,0,54.0,ST,Post Graduate,18477888.0,847000.0,1489790
2,Telangana,ADILABAD,0,INC,MALE,3,52.0,ST,12th Pass,36491000.0,15300000.0,1489790
4,Uttar Pradesh,AGRA,1,BJP,MALE,5,58.0,SC,Doctorate,74274036.0,8606522.0,1937690
5,Uttar Pradesh,AGRA,0,BSP,MALE,0,47.0,SC,Post Graduate,133784385.0,22251891.0,1937690


In [ ]:
data.shape

(2018, 12)

In [ ]:
data.isnull().sum()

STATE              0
CONSTITUENCY       0
WINNER             0
PARTY              0
GENDER             0
CRIMINAL\nCASES    0
AGE                0
CATEGORY           0
EDUCATION          0
ASSETS             0
LIABILITIES        0
TOTAL ELECTORS     0
dtype: int64

In [ ]:
data.dtypes

STATE               object
CONSTITUENCY        object
WINNER               int64
PARTY               object
GENDER              object
CRIMINAL\nCASES     object
AGE                float64
CATEGORY            object
EDUCATION           object
ASSETS             float64
LIABILITIES        float64
TOTAL ELECTORS       int64
dtype: object

In [ ]:
x = data.drop('WINNER',axis=1)

In [ ]:
y=data['WINNER']

In [ ]:
x = pd.get_dummies(x)

In [ ]:
x.shape

(2018, 758)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.30,random_state=42)

In [ ]:
model = SVC()

In [ ]:
# parameterssvc = {'kernal':['linear']}

In [ ]:
# Parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
#                      'C': [1, 10, 20]},
#                     {'kernel': ['linear'], 'C': [1, 10, 20]}]

In [ ]:
# parameters_new = [{'C':[0.001,0.1],'gamma':[0.1],'kernal':['rbf']}]

In [ ]:
Parameters = [{'kernel': ['rbf'], 'gamma': [0.1,0.2,0.5,0.75,1,2,3,4,5,6,7,8,9,10,15],
                     'C': [0.1,0.2,0.5,0.75,1,2,3,4,5,6,7,8,9,10,15]},
                    {'kernel': ['linear'], 'C': [1,10]}]

In [ ]:
grid_svc = GridSearchCV(model,Parameters,cv=5,n_jobs=-1)

In [ ]:
grid_svc.fit(x_train,y_train)

In [ ]:
model_LR = LogisticRegression()

In [ ]:
penalty = ['l1', 'l2', 'elasticnet']

In [ ]:
solver =  ['sag', 'saga']

In [ ]:
parameter_LR = {'solver':solver,'penalty':penalty} 

In [ ]:
grid_search = GridSearchCV(model_LR,parameter_LR,cv=5)

In [ ]:
grid_search.fit(x_train,y_train)

In [ ]:
grid_search.best_params_

In [ ]:
final_model = LogisticRegression(penalty = 'l2',solver='sag')

In [ ]:
final_model.fit(x_train,y_train)

In [ ]:
y_pred = final_model.predict(x_test)

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
classification_report(y_test,y_pred)